<a href="https://colab.research.google.com/github/scampion/geocodenet/blob/main/geocodenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/france.csv')
df.fillna('', inplace=True)
for c in ['number', 'street', 'postalcode', 'city']:    
    df[c].astype(object).replace(np.nan, 'None')
df['address'] = (df['number'] + " " + df['street'] + " " + df['postalcode'] + " " + df['city'])
del df['number']
del df['street']
del df['postalcode']
del df['city']
df['address'] = df['address'].str.lower()
df.head(n=10)

,Unnamed: 0,long,lat,address
0,0,0.504040,0.742304,31500 toulouse
1,1,0.490218,0.770736,0 chemin de halage 22300 lannion
2,2,0.490218,0.770736,22300 lannion
3,3,0.506738,0.771838,0 place des déportés 93350 le bourget
4,4,0.506738,0.771838,93350 le bourget
5,5,0.509069,0.776955,02100 saint-quentin
6,6,0.518043,0.747438,05200 baratier
7,7,0.509021,0.740731,1 avenue de la dévèze 34500 béziers
8,8,0.509021,0.740731,34500 béziers
9,9,0.516791,0.741160,83170 brignoles


In [11]:
max(df['address'].map(len)) # max address size
max_address_length = 200 

In [142]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
def str_to_indexes(s, length):
    s = s.lower()
    max_length = min(len(s), length)
    str2idx = np.zeros(length, dtype='int64')
    for i in range(1, max_length + 1):
        c = s[-i]
        if c in alphabet:
            str2idx[i - 1] = alphabet.index(c)
    return str2idx

In [144]:
df['address_encoded'] = df['address'].apply(str_to_indexes, length=max_address_length)

In [145]:
nbobs = len(df['address_encoded'])
X = np.concatenate(df['address_encoded'].to_numpy()).ravel()
X = X.reshape(nbobs, max_address_length)

In [48]:
X[0].shape

(200,)

In [15]:
Y = np.concatenate(df[['long', 'lat']].to_numpy()).ravel()
Y = Y.reshape(nbobs, 2)

In [30]:
import os 
try:
 device_name = os.environ['COLAB_TPU_ADDR']
 tpu_address = 'grpc://' + device_name
 print('Found TPU at: {}'.format(tpu_address))
except KeyError:
 print('TPU not found')

Found TPU at: grpc://10.111.13.178:8470


In [31]:
import tensorflow as tf

cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
tpu_strategy = tf.distribute.TPUStrategy(cluster_resolver)



INFO:tensorflow:Initializing the TPU system: grpc://10.111.13.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.111.13.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [113]:
import json 
config = json.load(open("./config.json"))
input_size=config["data"]["input_size"]
alphabet_size=config["data"]["alphabet_size"]
embedding_size=config["char_cnn_zhang"]["embedding_size"]
conv_layers=config["char_cnn_zhang"]["conv_layers"]
fully_connected_layers=config["char_cnn_zhang"]["fully_connected_layers"]
num_of_classes=config["data"]["num_of_classes"]
threshold=config["char_cnn_zhang"]["threshold"]
dropout_p=config["char_cnn_zhang"]["dropout_p"]
print("Dropout param: "+str(dropout_p))
optimizer=config["char_cnn_zhang"]["optimizer"]
loss=config["char_cnn_zhang"]["loss"]



Dropout param: 0.5


In [123]:
!pip install keras-metrics

In [136]:
from keras import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input, Dense, Flatten, Bidirectional, LSTM, Convolution1D, ThresholdedReLU, MaxPooling1D, Dropout
import keras_metrics

input_size = max_address_length
alphabet_size = len(alphabet)
embedding_size = 128
out_dim = 2

with tpu_strategy.scope():
  inputs = Input(shape=(input_size,), name='sent_input', dtype='int64')      
  x = Embedding(alphabet_size + 1, embedding_size, input_length=input_size)(inputs)
  for cl in conv_layers:
      x = Convolution1D(cl[0], cl[1])(x)
      x = ThresholdedReLU(threshold)(x)
      if cl[2] != -1:
          x = MaxPooling1D(cl[2])(x)
  x = Flatten()(x)
  for fl in fully_connected_layers:
      x = Dense(fl)(x)
      x = ThresholdedReLU(threshold)(x)
      x = Dropout(dropout_p)(x)
          
  predictions = Dense(2, activation='linear')(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer=optimizer, loss="mean_absolute_percentage_error", metrics=[keras_metrics.precision(), keras_metrics.recall()]) #, metrics=['accuracy'])
  print("CharCNNZhang model built: ")
  model.summary()

CharCNNZhang model built: 
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sent_input (InputLayer)      [(None, 200)]             0         
_________________________________________________________________
embedding_23 (Embedding)     (None, 200, 128)          8960      
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 194, 256)          229632    
_________________________________________________________________
thresholded_re_lu_73 (Thresh (None, 194, 256)          0         
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 64, 256)           0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 58, 256)           459008    
_________________________________________________________________
thresholded_re_lu_74 (Thre

In [ ]:
epochs=config["training"]["epochs"]
batch_size=config["training"]["batch_size"]
checkpoint_every=config["training"]["checkpoint_every"]
    
print("Training model: ")
model.fit(X, Y, epochs=epochs, batch_size=batch_size, callbacks=[])

Training model: 
Epoch 1/7
   1/6898 [..............................] - ETA: 5:01 - loss: 0.1581 - precision: 0.9782 - recall: 1.0000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0260s). Check your callbacks.


6898/6898 [==============================] - 166s 24ms/step - loss: 0.1743 - precision: 0.9811 - recall: 0.9887
Epoch 2/7
6898/6898 [==============================] - 168s 24ms/step - loss: 0.1752 - precision: 0.9812 - recall: 0.9891
Epoch 3/7
6898/6898 [==============================] - 170s 25ms/step - loss: 0.1765 - precision: 0.9799 - recall: 0.9875
Epoch 4/7
6898/6898 [==============================] - 175s 25ms/step - loss: 0.1774 - precision: 0.9801 - recall: 0.9881
Epoch 5/7
6898/6898 [==============================] - 175s 25ms/step - loss: 0.1810 - precision: 0.9776 - recall: 0.9872
Epoch 6/7
6898/6898 [==============================] - 172s 25ms/step - loss: 0.1965 - precision: 0.9695 - recall: 0.9868
Epoch 7/7
 698/6898 [==>...........................] - ETA: 2:37 - loss: 0.1862 - precision: 0.9755 - recall: 0.9848

In [147]:
q = "0 Chemin des Hautes Varennes 45200	Montargis".lower()
Q = str_to_indexes(q, max_address_length)
print(Q.shape)

print(Q)
for r in model.predict(np.array([Q])):
   print(r)
   print_geopoint(r)


(200,)
[18  8  6 17  0 19 13 14 12  0 26 26 28 31 30  0 18  4 13 13  4 17  0 21
  0 18  4 19 20  0  7  0 18  4  3  0 13  8 12  4  7  2  0 26  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[0.5067425 0.7666895]
2.4272918701171875 48.004106283187866


In [82]:
def print_geopoint(p):
  lon, lat = p
  print(lon * 360 - 180, lat * 180 - 90)

In [133]:
Y[0], print_geopoint(Y[0])

1.4543855000000008 43.61480230000012


(array([0.50403996, 0.74230446]), None)

In [109]:
 model.evaluate(X[0:20], Y[0:20], verbose=0)

0.28922078013420105